In [11]:
import requests
import json

import csv
import pandas as pd

import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#DL from here https://spotipy.readthedocs.io/en/2.21.0/
#pip install spotipy --upgrade

from langdetect import detect
#DL from here https://pypi.org/project/langdetect/
#pip install langdetect

In [12]:
#First you will need to download the 5 million song dataset.
#the zip alone is 3.1 gigs while the actual dataset is like 9.21

#https://www.kaggle.com/datasets/nikhilnayak123/5-million-song-lyrics-dataset?resource=download

In [13]:
#Your name?

#name = 'terrence'
#name = 'yenho'
#name = 'gabriela'
name = 'terrDesk'
#name = 'gabDesk'

In [18]:
#PARTITION THE DATASET

mypath = '../../ds2.csv'

In [19]:
if name == 'terrence':
    df = pd.read_csv(mypath, nrows=1000000)

elif name == 'yenho':
    df = pd.read_csv(mypath, skiprows=(1, 1000000), nrows=800080)
    
elif name == 'gabriela':
    df = pd.read_csv(mypath, skiprows=(1, 2000000), nrows=800080)
    
elif name == 'terrDesk':
    df = pd.read_csv(mypath, skiprows=(1, 3000000), nrows=800080)
    
elif name == 'gabDesk':
    df = pd.read_csv(mypath, skiprows=(1, 4000000), nrows=800080) #for Gabriela's desktop

In [21]:
df.head()

,μs Muse - Snow Halation English Translation,pop,Genius English Translations,2010,10254,{},"This feeling, it's strange, isn't it?\nIt's almost as though it came fluttering down from the sky\nThe color of this special season fills me with excitement\n\nFrom the moment we met\nThe melody of my heart has been ringing with premonitions\nIt can’t be stopped, it won’t be stopped -- why?\n\nRing out\nShall I name this heart-rending pain ""Snow halation""?\nI can’t wait for our feelings to resonate; it’s frustrating, but it’s a pure-hearted devotion called love\nEven in this mild fever, I can’t hesitate\nI'll accept the courage to dive in; it's about to start!\n\n\nWithout a sound, nor a sign\nOur fate changes silently\nAt the thought of the coming future, my heartbeat quickens\n\nWhen you feel troubled\nI’ll come running right away, wanting to hug you tightly\nWherever you are, no matter where, I'll Fly high!\nHurry up\nBefore I knew it, my true emotions grew too large to contain\nIt's painful to never do more than dream of it\nI want you to be mine!\nI don't want to be bewildered by your gentle eyes\nAccept all my love for you as it is, Please!\n\n\nRing out\nShall I name this heart-rending pain ""Snow halation""?\nI can’t wait for our feelings to resonate; It’s frustrating, but it’s a pure-hearted devotion called love\nEven in this mild fever, I can’t hesitate\nI'll accept the courage to dive in; it's about to start!",4217918
0,King Size,rap,Young Thug,2019,1518,"{""Swizz Beatz"",""Lil Keed""}",Lyrics from Snippets\n\n[Intro: Young Thug]\nY...,4217919
1,Awake,pop,Tourist,2019,246,{},[Instrumental],4217920
2,Midnight Traffic,rap,Dramah,2018,36,{},Part 1\nNachts ist wieder meine Zeit\nUnd ich ...,4217921
3,Flor da Vida. Interlúdio,misc,Freelogia,2019,52,"{""Quadro em Branco""}",[Verso: Otavio Oliveira]\nA vida é como se fos...,4217922
4,Emily,pop,Tourist,2019,3021,{},"[Intro]\nAnd there's just, you know, days when...",4217924


In [20]:
#filter out nan values
df = df[df['lyrics'] == df['lyrics']]

KeyError: 'lyrics'

In [ ]:
#Filter out non english songs. This section will take a while.

def lang(row):
    lyrics = row['lyrics']
    
    try:
        lang = detect(lyrics)
    except:
        #print(row)
        #print(type(lyrics))
        lang = 'bs'
            
    return lang

df['lang'] = df.apply(lambda row : lang(row), axis = 1)

df = df[df['lang'] == 'en']

In [ ]:
#Process lyrics

def lyricEdit(row):
    lyrics = row['lyrics']

    lyrics = re.sub("[\[].*?[\]]", "", lyrics)

    lyrics = re.sub(r'\n\s*\n', '\n', lyrics)
    
    lyrics = lyrics.split('\n')

    if lyrics[0] == '':
            lyrics = lyrics[1:]
            
    return lyrics

df['lyrics'] = df.apply(lambda row : lyricEdit(row), axis = 1)

In [ ]:
df.shape #from a million, i was able to still have 786k after all this filtering.

In [ ]:
def swapKeys(user, idx):
    while True:
        cidLst = ["b7b309716161449da86485a6348a346e", 
                  "6c401ae9ee5f4b5286249499b2633ead",
                  "758e3d5b94a14815ae6d336d7e37b209",
                  "0da9f55d56884f99869265490b334326"]

        if user == "terrence":
            cidLst.append("715ae75406e24c24a3641929260ca343")
        elif user == "yenho":
            cidLst.append("e3fc9e837db6490184d4095450c3508f")
        elif user == "gabriela":
            cidLst.append("7eabd16573dc4a23bbb90b2e6578ba9d")
        elif user == "terrDesk":
            cidLst.append("4b387b6ec9d6446db9035cb56d744fec")
        elif user == 'gabDesk':
            cidLst = cidLst[::-1]
            cidLst.append("0daf9f613d0946d780ebfb062c0d8ad7")

        secretLst = ["8d8dea83579b4501b0526f45c671c94b", 
                     "ef9cfc951ddd4913986588b761a1a55d",
                     "5448a297b3ba4ebd907eb2d2657ea15b",
                     "be709045ba894650952c1733af4d201e"]

        if user == "terrence":
            secretLst.append("e60a686258c646b7a4d3f9636b3235db")
        elif user == "yenho":
            secretLst.append("493edd8594fa4509b1ea52c7a1faae5c")
        elif user == "gabriela":
            secretLst.append("732827c5c1a2499a853c12ec73ca5264")
        elif user == "terrDesk":
            secretLst.append("dbeee0427c8f41dd99f5703b157ee252")
        elif user == 'gabDesk':
            secretLst = secretLst[::-1]
            secretLst.append("3b3245dde0404a6b87906235d9a9971a")

        cid = cidLst[idx]
        secret = secretLst[idx]

        idx += 1

        if idx == len(cidLst):
            idx = 0

        AUTH_URL = "https://accounts.spotify.com/api/token"
        
        try:
            auth_response = requests.post(AUTH_URL, {
                'grant_type': 'client_credentials',
                'client_id': cid,
                'client_secret': secret,
            })
        except:
            print('\nHaving issues connecting... trying again.')
            continue

        try:
            auth_response_data = auth_response.json()
        except:
            print('\nHaving issues connecting... trying again.')
            continue

        access_token = auth_response_data['access_token']

        headers = {
            'Authorization': 'Bearer {token}'.format(token=access_token)
        }

        return headers, idx


In [ ]:
def getUri(row, idx):
    headers, _ = swapKeys(name, idx)
    
    artistName = row['artist']
    trackName = row['title']

    artistName = re.sub("[\(\[].*?[\)\]]", "", artistName)
    trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)

    trackName2 = trackName.replace(' ', '%20')
    artistName2 = artistName.replace(' ', '%20')
    
    trackName = trackName.upper()
    trackName = trackName.replace("'", "")
    trackName = trackName.replace("&", "")
    trackName = trackName.replace(" ", "")
    
    artistName = artistName.upper()
    artistName = artistName.replace("'", "")
    artistName = artistName.replace("&", "")
    artistName = artistName.replace(" ", "")
    
    attempts = 0
    trying = True
    
    while(trying):
        BASE_URL = 'https://api.spotify.com/v1/'
        r = requests.get(BASE_URL + 'search?q=' + artistName2 + '%20track:' + trackName2 + '&type=track', headers=headers)
        
        while(True):
            try:
                r = r.json()
                break
            except:
                #if str(r)[-5:-2] == "429":
                print("\tIt seems rate limit is reached, swapping keys")
                headers, idx = swapKeys(name, idx)
                r = requests.get(BASE_URL + 'search?q=' + artistName2 + '%20track:' + trackName2 + '&type=track', headers=headers)
    
        if 'tracks' in r:
            break
        else:
            print('\nTracks not found in r, trying again and swapping gang')
            attempts += 1
            headers, idx = swapKeys(name, idx)
            
            if attempts == 10:
                trying = False
            
            print('\t',r)
            
    if trying == False:
        print('\tcouldnt get a search. skipping for now',artistName,'|',trackName)
        return '', idx

    if r['tracks']['items'] == []:
        print('\tno search results found:',artistName,'|',trackName)
        return '', idx

    for item in r['tracks']['items']:
        editedName = item['name'].upper()
        editedName = editedName.replace("'", "")
        editedName = editedName.replace("&", "")
        editedName = editedName.replace(" ", "")
        
        editedArt = item['artists'][0]['name'].upper()
        editedArt = editedArt.replace("'", "")
        editedArt = editedArt.replace("&", "")
        editedArt = editedArt.replace(" ", "")
        
        trackCorr = (editedName == trackName) or (editedName in trackName) or (trackName in editedName)
        artCorr = (editedArt == artistName) or (editedArt in artistName) or (artistName in editedArt)
        
        if (trackCorr or artCorr):
            print('\tgottem:',artistName,'|',trackName)
            return item['uri'], idx

    print('\tnone of the items matched:',artistName,'|',trackName)
    return '', idx
        

In [ ]:
#UNCOMMENT THIS THEN RUN IT. AFTER YOU RUN IT, COMMENT IT AGAIN
#BECAUSE YOU CANNOT RISK RUNNING THIS CELL AGAIN IF YOURE DEEP INTO RECORDING URIS (IT WILL RESET THE CSV AND LIST)
'''
uriLst = []

realUris = 0

#writing URIs to a csv just to save them somewhere :)
with open('uri.csv', 'w', newline='') as fCsv:
    writer = csv.writer(fCsv)

    header = ['uri']
    writer.writerow(header)

'''

In [ ]:
idx = 2
with open('uri.csv', 'a', newline='') as fCsv:
    writer = csv.writer(fCsv)
    
    for index, row in df.iloc[len(uriLst):].iterrows():
        if realUris >= 200020:
            break
            
        print('\nOn row:',len(uriLst), '| # of real URIs obtained:',realUris)
        uri, idx = getUri(row, idx)
        
        if uri != '':
            realUris += 1
        
        uriLst.append(uri)
        
        writer.writerow([uri])
#'''

In [ ]:
df['uri'] = uriLst
df = df[df['uri'] != '']

df.shape

In [ ]:
if name == 'terrence':
    df.to_csv('musicData.csv', index=False)
    
elif name == 'yenho':
    df.to_csv('musicData2.csv', index=False)
    
elif name == 'gabriela':
    df.to_csv('musicData3.csv', index=False)
    
elif name == 'terrDesk':
    df.to_csv('musicData4.csv', index=False)
    
elif name == 'gabDesk':
    df.to_csv('musicData5.csv', index=False)